# 1.1) Analisando os Tipos de Dados

In [ ]:
# Importações
!pip install lazypredict
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split
# Alterando a vizualização(vai ser relevante futuramente)
pd.set_option('display.max_columns', None)
# Importando o data frame dos inscritos
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1CC4UlTaP7hPyWdtNLOTm2hZLTTduxiVk')
df

,id,age,job,marital,education,location,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,40000,26.00,self-employed,single,tertiary,Porto,no,-101.00,no,yes,cellular,29,jan,172,1.00,-1,0.00,unknown
1,40001,47.00,technician,married,secondary,NaN,no,1167.00,yes,no,cellular,2,feb,139,3.00,73,2.00,success
2,40002,35.00,self-employed,married,tertiary,NaN,no,2427.00,no,no,cellular,12,jul,210,3.00,158,7.00,other
3,40003,54.00,technician,divorced,secondary,NaN,no,NaN,no,no,cellular,30,jan,86,1.00,-1,0.00,unknown
4,40004,27.00,blue-collar,single,secondary,Braga,no,-208.00,yes,n,unknown,21,may,127,2.00,-1,0.00,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206,45206,38.00,administrator,married,secondary,NaN,no,1767.00,yes,no,cellular,12,nov,893,7.00,-1,0.00,unknown
5207,45207,39.00,housemaid,married,primary,Lisboa,no,562.00,no,no,cellular,12,aug,234,2.00,-1,0.00,unknown
5208,45208,46.00,technician,married,secondary,Amadora,no,0.00,no,no,unknown,20,jun,625,1.00,-1,0.00,unknown
5209,45209,NaN,technician,single,secondary,Braga,no,1195.00,yes,no,cellular,19,nov,187,6.00,188,9.00,failure


Vamos começar pegando as categorias que só possuem valores binários e transformando em numéricos para facilitar a análise

In [ ]:
replacer = { 'y' : 1, 'yes': 1, 'n' : 0 , 'no' : 0}
df[['default','housing','loan']] = df[['default','housing','loan']].replace(replacer)


# 1.2) Limpeza do Dataset

O id, como esperado não tem correlação com nada então vamos começar removendo-o, além disso, obtivemos algumas correlações entre features com alto grau porém que não estão diretamente ligadas ao target, vamos deixar alguns gráficos que registram essa correlação apenas por desencargo de consiência. Por fim vamos também tratar os dados faltantes no meio do caminho.

In [ ]:
identificacao = df['id']
df.drop(columns = 'id', axis = 1, inplace = True)

In [ ]:
df.drop(columns = 'location', axis = 1, inplace = True)

In [ ]:
df['job'].unique()
df['job'] = df['job'].replace({"admin." : "administrator"})

Remoção de Outliers

In [ ]:
indices_para_substituir = df[df['balance'] > 50000].index.tolist()

# Exibir os valores e índices para verificação
print("Valores:", df.loc[indices_para_substituir, 'balance'].tolist())
print("Índices:", indices_para_substituir)

#Remover as linhas pelos índices
df.loc[indices_para_substituir, 'balance'] = np.nan

# Redefinir os índices do DataFrame
df.reset_index(drop=True, inplace=True)

Valores: [71188.0, 51439.0, 58932.0]
Índices: [560, 1881, 5132]


In [ ]:
indices_para_substituir = df[df['age'] > 100].index.tolist()

# Exibir os valores e índices para verificação
print("Valores:", df.loc[indices_para_substituir, 'age'].tolist())
print("Índices:", indices_para_substituir)

# Remover as linhas pelos índices
df.loc[indices_para_substituir, 'age'] = np.nan

# Redefinir os índices do DataFrame
df.reset_index(drop=True, inplace=True)


Valores: [150.0, 150.0, 150.0]
Índices: [524, 2976, 4599]


In [ ]:
indices_para_substituir = df[df['duration'] > 5000].index.tolist()

# Exibir os valores e índices para verificação
print("Valores:", df.loc[indices_para_substituir, 'duration'].tolist())
print("Índices:", indices_para_substituir)

# Remover as linhas pelos índices
df.loc[indices_para_substituir, 'duration'] = np.nan

# Redefinir os índices do DataFrame
df.reset_index(drop=True, inplace=True)

Valores: [75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000, 75000]
Índices: [221, 504, 716, 922, 1199, 1431, 2045, 2196, 2915, 3319, 3619, 3664, 3681, 4297, 4634]


In [ ]:
indices_para_substituir = df[df['campaign'] > 35].index.tolist()

# Exibir os valores e índices para verificação
print("Valores:", df.loc[indices_para_substituir, 'campaign'].tolist())
print("Índices:", indices_para_substituir)

# Remover as linhas pelos índices
df.loc[indices_para_substituir, 'campaign'] = np.nan


# Redefinir os índices do DataFrame
df.reset_index(drop=True, inplace=True)

Valores: [51.0, 46.0, 63.0, 38.0, 44.0, 43.0, 50.0]
Índices: [370, 638, 2452, 2758, 3233, 4484, 5191]


In [ ]:
indices_para_substituir = df[df['previous'] > 30].index.tolist()

# Exibir os valores e índices para verificação
print("Valores:", df.loc[indices_para_substituir, 'previous'].tolist())
print("Índices:", indices_para_substituir)

# Remover as linhas pelos índices
df.loc[indices_para_substituir, 'previous'] = np.nan

# Redefinir os índices do DataFrame
df.reset_index(drop=True, inplace=True)

Valores: [38.0, 51.0, 37.0]
Índices: [1742, 3566, 4953]


In [ ]:
indices_para_substituir = df[df['pdays'] > 700].index.tolist()

# Exibir os valores e índices para verificação
print("Valores:", df.loc[indices_para_substituir, 'pdays'].tolist())
print("Índices:", indices_para_substituir)

# Remover as linhas pelos índices
df.loc[indices_para_substituir, 'pdays'] = np.nan

# Redefinir os índices do DataFrame
df.reset_index(drop=True, inplace=True)

Valores: [745, 701, 842, 805]
Índices: [407, 1264, 2232, 4175]


Analisando se será preciso um oversampling ou undersampling no futuro


In [ ]:
categoricas = np.array(("job", "month", "contact", "marital", "education"))
numericas = np.array(("age", "balance","duration","campaign","day"))

Deletando colunas sem uma correlação grande com o target

In [ ]:
df.drop(columns = ['marital','contact'], axis = 1, inplace = True)

In [ ]:
df.replace('unknown', np.nan, inplace=True)

Não temos como trabalhar com a coluna de poutcome pois a quantidade de dados que não temos supera em muito a quantidade de dados que temos, portanto vamos excluí-la.

In [ ]:
df.drop(columns = 'poutcome', axis = 1, inplace = True)

In [ ]:
replacer2 = { 'primary': 0, 'secondary': 0.5, 'tertiary' : 1}
df['education'] = df['education'].replace(replacer2)

Vamos agora aplicar one hot-encoding nas devidas categorias

In [ ]:
# Selecionar as colunas categóricas
categorical_columns = ['job', 'month']

# Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(df[categorical_columns], columns=categorical_columns)
#binarizando
df_encoded = df_encoded.astype(int)
df_encoded.head()


,job_administrator,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


Vamos normalizar as features numéricas

In [ ]:
import joblib

normalizando = ['education', 'age', 'balance', 'duration', 'campaign', 'day','pdays','previous']
scaler = joblib.load('scaler (2).pkl')
norm = scaler.fit_transform(df[normalizando])

# Convertendo o array numpy de volta para um DataFrame do Pandas
df_norm = pd.DataFrame(norm, columns=normalizando)


In [ ]:
#Reformatando o dataframe
df.drop(columns = categorical_columns, axis = 1, inplace = True)
df.drop(columns = normalizando, axis = 1, inplace = True)

df_aux = pd.concat([df_encoded, df_norm], axis=1)
df = pd.concat([df_aux, df], axis=1)

df.head()

,job_administrator,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,education,age,balance,duration,campaign,day,pdays,previous,default,housing,loan
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1.00,0.23,0.05,0.06,0.00,0.93,0.00,0.00,0,0,1
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.50,0.49,0.08,0.05,0.06,0.03,0.12,0.07,0,1,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.00,0.34,0.12,0.07,0.06,0.37,0.25,0.24,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.50,0.58,NaN,0.03,0.00,0.97,0.00,0.00,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.50,0.24,0.05,0.04,0.03,0.67,0.00,0.00,0,1,0


Como nosso dataset é relativamente pequeno e já tratamos manualmente casos mais extremos de outliers ou colunas problemáticas podemos terminar o serviço de preenchimento de valores faltantes com o método KNN, que trará para esse conjunto de dados relativamente comportado, conforme vimos nos gráficos anteriores, uma boa estimativa.

In [ ]:
# Carregar o KNNImputer salvo
imputer = joblib.load('knn_imputer (1) (4).pkl')

# Imputar os dados faltantes no novo conjunto
new_data_imputed = imputer.transform(df)

# Converter de volta para DataFrame
# Certifique-se de que df está definido e tem as colunas esperadas
df = pd.DataFrame(new_data_imputed, columns=df.columns)

In [ ]:
df.tail()

,job_administrator,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,education,age,balance,duration,campaign,day,pdays,previous,default,housing,loan
5206,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.50,0.38,0.10,0.27,0.18,0.37,0.00,0.00,0.00,1.00,0.00
5207,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.39,0.07,0.07,0.03,0.37,0.00,0.00,0.00,0.00,0.00
5208,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.50,0.48,0.05,0.19,0.00,0.63,0.00,0.00,0.00,0.00,0.00
5209,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.50,0.37,0.08,0.06,0.15,0.60,0.30,0.31,0.00,1.00,0.00
5210,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.50,0.61,0.17,0.05,0.00,0.20,0.00,0.00,0.00,1.00,0.00


De 40000 dados perdemos apenas 255 na limpeza, o que representa 0,64% de perda de informação apenas, não irá afetar o resultado do treinamento.

#1.4) Preparando o modelo para exportação



In [ ]:
from google.colab import drive



In [ ]:
df.to_csv('dataset_limpo_teste.csv', index=False)

# 1.5) Usando o Modelo Exportado para achar os resultados

In [ ]:
# Carregar o modelo salvo
loaded_model = joblib.load('best_model_lgbm (2).pkl')

# Fazer previsões de probabilidade
probabilidades = loaded_model.predict_proba(df)


# Fazer previsões
y_pred = loaded_model.predict(df)

# Criar um DataFrame com as colunas "id" e "categoria"
resultado = pd.DataFrame({"id": identificacao, "y": y_pred})

In [ ]:
resultado.head()

,id,y
0,40000,0
1,40001,1
2,40002,1
3,40003,0
4,40004,0


In [ ]:
resultado.to_csv('resposta_do_Projeto.csv', index=False)